In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../NVDA.csv")

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022/7/1,14.899,15.063000,14.392,14.523,14.506663,577610000.0
1,2022/7/5,14.175,14.971000,14.055,14.964,14.947166,651397000.0
2,2022/7/6,15.010,15.319000,14.789,15.130,15.112980,529066000.0
3,2022/7/7,15.456,15.945000,15.389,15.858,15.840160,492903000.0
4,2022/7/8,15.430,16.037001,15.389,15.838,15.820185,467972000.0
